In [5]:
!pip install seaborn


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("GlobalLandTemperaturesByCity.csv")


In [3]:
print("First 5 rows:")
print(df.head(5), "\n")
print("last 5 rows")
print(df.tail(5), "\n")
print("information of data:")
print(df.info(), "\n")
print("Description of data:")
print(df.describe(), "\n")
print("Data shape:")
print(df.shape)


First 5 rows:
           dt  AverageTemperature  AverageTemperatureUncertainty   City  \
0  1743-11-01               6.068                          1.737  Århus   
1  1743-12-01                 NaN                            NaN  Århus   
2  1744-01-01                 NaN                            NaN  Århus   
3  1744-02-01                 NaN                            NaN  Århus   
4  1744-03-01                 NaN                            NaN  Århus   

   Country Latitude Longitude  
0  Denmark   57.05N    10.33E  
1  Denmark   57.05N    10.33E  
2  Denmark   57.05N    10.33E  
3  Denmark   57.05N    10.33E  
4  Denmark   57.05N    10.33E   

last 5 rows
                 dt  AverageTemperature  AverageTemperatureUncertainty  \
8599207  2013-05-01              11.464                          0.236   
8599208  2013-06-01              15.043                          0.261   
8599209  2013-07-01              18.775                          0.193   
8599210  2013-08-01              

In [10]:
print("missing values")
print(df.isna().sum())
print("\nduplicated values")
print(df.duplicated().sum())
df['AverageTemperature'] = df['AverageTemperature'].fillna(df['AverageTemperature'].mean())
df['AverageTemperatureUncertainty'] = df['AverageTemperatureUncertainty'].fillna(df['AverageTemperatureUncertainty'].mean())
print("\nData after filling missing values:")
print(df.isna().sum())
df_cleaned = df.dropna()
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned = df_cleaned.reset_index(drop=True)
print("\nMissing values after cleaning:")
print(df_cleaned.isna().sum(), "\n")

missing values
dt                                    0
AverageTemperature               364130
AverageTemperatureUncertainty    364130
City                                  0
Country                               0
Latitude                              0
Longitude                             0
dtype: int64

duplicated values
0

Data after filling missing values:
dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64

Missing values after cleaning:
dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64 



In [11]:
print("All available columns:\n", df_cleaned.columns, "\n")
selected_features = ['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City', 'Country']
df_selected = df_cleaned[selected_features].copy()
#'dt' to datetime format for time-based analysis
df_selected['dt'] = pd.to_datetime(df_selected['dt'], errors='coerce')
df_selected = df_selected.dropna(subset=['dt'])
df_selected = df_selected.reset_index(drop=True)
print(df_selected.info(), "\n")
print(df_selected.isna().sum())

All available columns:
 Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 5 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   dt                             datetime64[ns]
 1   AverageTemperature             float64       
 2   AverageTemperatureUncertainty  float64       
 3   City                           object        
 4   Country                        object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 328.0+ MB
None 

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
dtype: int64


In [5]:
print("Basic Descriptive Statistics:\n")
print(df_selected.describe())

print("\nAverage Temperature Distribution")
plt.figure(figsize=(8,5))
sns.histplot(df_selected['AverageTemperature'], bins=40, kde=True, color='skyblue')
plt.title('Distribution of Average Temperature')
plt.xlabel('Average Temperature (°C)')
plt.ylabel('Frequency')
plt.show()

print("\n\nAverage Temperature Uncertainty Distribution")
plt.figure(figsize=(8,5))
sns.histplot(df_selected['AverageTemperatureUncertainty'], bins=40, kde=True, color='orange')
plt.title('Distribution of Temperature Uncertainty')
plt.xlabel('Uncertainty (°C)')
plt.ylabel('Frequency')
plt.show()

print("\n\nTop 10 Countries by Average Temperature")
top_countries = (
    df_selected.groupby('Country')['AverageTemperature'].mean().sort_values(ascending=False).head(10)
)
plt.figure(figsize=(10,5))
sns.barplot(x=top_countries.values, y=top_countries.index, palette='coolwarm')
plt.title('Top 10 Countries by Average Temperature')
plt.xlabel('Average Temperature (°C)')
plt.ylabel('Country')
plt.show()

print("\n\nGlobal Average Temperature Trend Over Years")
df_trend = (
    df_selected.groupby(df_selected['dt'].dt.year)['AverageTemperature']
    .mean()
    .reset_index()
)
plt.figure(figsize=(10,5))
plt.plot(df_trend['dt'], df_trend['AverageTemperature'], color='darkblue', linewidth=2)
plt.title('Global Average Temperature Trend Over Years')
plt.xlabel('Year')
plt.ylabel('Average Temperature (°C)')
plt.grid(True)
plt.show()

print("\n\nCorrelation Between Temperature and Uncertainty")
plt.figure(figsize=(5,4))
sns.heatmap(df_selected[['AverageTemperature', 'AverageTemperatureUncertainty']].corr(),
            annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Between Temperature and Uncertainty')
plt.show()


Basic Descriptive Statistics:



NameError: name 'df_selected' is not defined

In [6]:
!pip install gradio matplotlib pandas scikit-learn


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# =========================
# 1. Load and prepare data
# =========================

CSV_PATH = "GlobalLandTemperaturesByCity.csv"  # تأكد الاسم صح

df = pd.read_csv(CSV_PATH)

# تأكد من التاريخ
if "dt" in df.columns:
    df["dt"] = pd.to_datetime(df["dt"], errors="coerce")

# أعمدة زمنية
if "year" not in df.columns and "dt" in df.columns:
    df["year"] = df["dt"].dt.year
if "month" not in df.columns and "dt" in df.columns:
    df["month"] = df["dt"].dt.month

# نتأكد إن درجة الحرارة موجودة ونشيل NaN عشان ما تلخبط الكلاستر
df = df.dropna(subset=["AverageTemperature"])

# قوائم الدول والمدن
countries = sorted(df["Country"].dropna().unique().tolist())
cities = sorted(df["City"].dropna().unique().tolist())

# =========================
# 1.1 Build climate clusters (cluster_label)
# =========================

# نخفف البيانات شوي: نخلي السنوات الحديثة (اختياري، تقدر تشيل الفلتر لو حاب)
recent = df[df["year"] >= 1900].copy()

# متوسط درجة الحرارة لكل (مدينة, دولة, شهر)
monthly_city = (
    recent
    .groupby(["City", "Country", "month"])["AverageTemperature"]
    .mean()
    .reset_index()
)

# pivot: صف لكل مدينة، 12 عمود = شهور
pivot = monthly_city.pivot_table(
    index=["City", "Country"],
    columns="month",
    values="AverageTemperature"
)

# نحذف المدن اللي ناقصها شهور كثيرة
pivot = pivot.dropna(how="any")

# Scaling + KMeans
scaler = StandardScaler()
X_clust = scaler.fit_transform(pivot.values)

k = 3  # عدد الكلاسترز
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_clust)

pivot["cluster_label"] = [f"Cluster_{c}" for c in clusters]

# نخليها جدول (City, Country, cluster_label) ونرجعها للـ df الأساسي
clusters_df = pivot[["cluster_label"]].reset_index()
df = df.merge(clusters_df, on=["City", "Country"], how="left")

print("cluster_label added. Value counts:")
print(df["cluster_label"].value_counts(dropna=False).head())

# =========================
# 2. Helper functions
# =========================

def plot_yearly_trend(selected_country, selected_city):
    """
    يرسم ترند متوسط الحرارة السنوي.
    لو المستخدم اختار مدينة -> نفلتر بالمدينة،
    لو اختار دولة -> نفلتر بالدولة،
    لو تركها All -> على كل البيانات.
    """
    data = df.copy()

    if selected_city and selected_city != "All":
        data = data[data["City"] == selected_city]
    elif selected_country and selected_country != "All":
        data = data[data["Country"] == selected_country]

    if "year" not in data.columns:
        return None, "No 'year' column available to compute trend."

    yearly = (
        data.groupby("year")["AverageTemperature"]
        .mean()
        .reset_index()
        .sort_values("year")
    )

    if yearly.empty:
        return None, "No data available for the selected filters."

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(yearly["year"], yearly["AverageTemperature"], marker="o")
    ax.set_title("Yearly Average Temperature Trend", fontsize=12)
    ax.set_xlabel("Year")
    ax.set_ylabel("Average Temperature (°C)")
    ax.grid(True)

    text = (
        f"Data points: {len(yearly)} years.\n"
        f"Showing average temperature trend based on the selected filters."
    )

    return fig, text


def plot_city_timeseries(selected_city):
    """
    يرسم سلسلة زمنية لدرجة الحرارة لمدينة معيّنة.
    """
    if not selected_city or selected_city == "All":
        return None, "Please select a specific city."

    if "dt" not in df.columns:
        return None, "No 'dt' (date) column available for time series."

    data = df[df["City"] == selected_city].copy().sort_values("dt")

    if data.empty:
        return None, "No data available for this city."

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(data["dt"], data["AverageTemperature"], linewidth=0.8)
    ax.set_title(f"Temperature Time Series – {selected_city}", fontsize=12)
    ax.set_xlabel("Date")
    ax.set_ylabel("Average Temperature (°C)")
    ax.grid(True)

    text = (
        f"City: {selected_city}\n"
        f"Number of records: {len(data)}.\n"
        "This plot shows how the average temperature changes over time for the selected city."
    )

    return fig, text


def plot_clusters():
    """
    يعرض توزيع المدن حسب cluster_label.
    """
    if "cluster_label" not in df.columns:
        return None, "No 'cluster_label' column found. Please run clustering first."

    city_clusters = (
        df[["City", "Country", "cluster_label"]]
        .drop_duplicates()
        .dropna(subset=["cluster_label"])
    )

    if city_clusters.empty:
        return None, "No cluster data available."

    cluster_counts = (
        city_clusters.groupby("cluster_label")["City"]
        .nunique()
        .reset_index(name="num_cities")
    )

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.bar(cluster_counts["cluster_label"], cluster_counts["num_cities"])
    ax.set_title("Number of Cities per Climate Cluster", fontsize=12)
    ax.set_xlabel("Cluster Label")
    ax.set_ylabel("Number of Cities")
    ax.grid(axis="y")

    text_lines = [f"{row['cluster_label']}: {row['num_cities']} cities"
                  for _, row in cluster_counts.iterrows()]
    text = "Cluster summary:\n" + "\n".join(text_lines)

    return fig, text


# =========================
# 3. Gradio UI
# =========================

def build_demo():
    with gr.Blocks() as demo:
        gr.Markdown(
            """
            # 🌍 Sustainable Climate Analytics – Dashboard  
            **CS316 – Phase 3 – Group 4**

            This web app visualizes the main results of our project:
            - Yearly warming trend (H1)  
            - City-level temperature time series (H2 context)  
            - Climate clusters for cities (H3)  
            """
        )

        with gr.Tab("Yearly Trend (H1)"):
            gr.Markdown("### Filter by Country/City (optional)")

            country_dd = gr.Dropdown(
                choices=["All"] + countries,
                value="All",
                label="Country"
            )

            city_dd = gr.Dropdown(
                choices=["All"] + cities,
                value="All",
                label="City"
            )

            trend_btn = gr.Button("Show Trend")
            trend_plot = gr.Plot(label="Yearly Average Temperature")
            trend_text = gr.Textbox(label="Summary", lines=4)

            trend_btn.click(
                fn=plot_yearly_trend,
                inputs=[country_dd, city_dd],
                outputs=[trend_plot, trend_text],
            )

        with gr.Tab("City Time Series (H2 Context)"):
            gr.Markdown("### Select a city to view its temperature over time")

            city_ts_dd = gr.Dropdown(
                choices=cities,
                value=cities[0] if cities else None,
                label="City"
            )

            ts_btn = gr.Button("Show Time Series")
            ts_plot = gr.Plot(label="City Temperature Time Series")
            ts_text = gr.Textbox(label="Summary", lines=4)

            ts_btn.click(
                fn=plot_city_timeseries,
                inputs=[city_ts_dd],
                outputs=[ts_plot, ts_text],
            )

        with gr.Tab("Climate Clusters (H3)"):
            gr.Markdown(
                "### Climate-based Clusters\n"
                "This view shows how many cities fall into each cluster_label "
                "(e.g., cold / moderate / hot climates)."
            )

            clus_btn = gr.Button("Show Cluster Overview")
            clus_plot = gr.Plot(label="Cities per Cluster")
            clus_text = gr.Textbox(label="Summary", lines=6)

            clus_btn.click(
                fn=plot_clusters,
                inputs=None,
                outputs=[clus_plot, clus_text],
            )

    return demo


demo = build_demo()


demo.launch()



/home/abdulaziz21/ai-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cluster_label added. Value counts:
cluster_label
Cluster_1    3045547
Cluster_2    2619162
Cluster_0    2570373
Name: count, dtype: int64
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
